In [2]:
#Topics covered
#Memory references
#What variables really are 
# memory management
# reference counting and garbage collection
#Dynamic and static typing
#Mutability and immutability
# Shared references!
#Variable equality
#Everything is an object including Classes themselves!

In [3]:
#Variables and Memory references!

In [5]:
# Variable are Memory references!
#Check more on heap memory!
# Storing and retrieving objects from heap is taken care of 
#by the python memory manager!

In [9]:
my_var1 = 10
#creates an object in memory  at address say 1000 and stores it there!

In [10]:
# my_var1 is a reference to 10, it is a pointer aka memory address of 10

In [11]:
my_var2 = 'hello'

In [12]:
#Variables in python are always references!

In [13]:
# In python we can find out the memory address referenced by a variable by using the id()
# function!

In [14]:
# id function will return a base 10 , and we can convert it into hex

In [28]:
#eg:
#a = 10
#print(hex(id(a)))

In [29]:
my_var = 10

In [30]:
print(my_var)

10


In [31]:
print(id(my_var))

1368812896


In [33]:
print(hex(id(my_var)))  # hexadecimal version of the address!

0x51966d60


In [34]:
greeting  = 'Hellooo'
print(greeting)

Hellooo


In [38]:
print(id(greeting))

2434899065968


In [39]:
print(hex(id(greeting)))

0x236eb4b3c70


### Reference Counting!

In [43]:
my_var = 11
# my_var is the reference/pointer to the object 11 created in the memory!

In [45]:
# How many variables are pointing to the same object?
# Here its 1 because no other variables refer to it !

In [49]:
other_var = my_var
# we are dealing with pointers!
#Now other_var still points to my_var
#Reference is shared, therefore reference count is 2

In [50]:
# if  no references are there
# Then the object is thrown away!

In [53]:
# Finding the Reference Count!
# sys.getrefcount (  <pass a variable> )
# passing my_var to getrefcount() creates an extra reference!

In [54]:
# There is another function that we can use without extra counting

In [55]:
# ctypes.c_long.from_address(address).value
# here we just pass the memory address, as we dont pass value!

In [56]:
import sys

In [64]:
a = [1, 2, 3, 4, 5, 6, 8, 9, 10 ]

In [65]:
id(a)

2434899180744

In [68]:
sys.getrefcount(a)
# See ref count is 2!  because the passing of  variable "a" caused this!

2

In [69]:
import ctypes

In [70]:
def ref_count(address : int):
    return ctypes.c_long.from_address(address).value

In [74]:
ref_count(id(a)) # we have to pass the address!
# But we still passed "a" inside the function
#id(a) is getting evaulated first
# and as id finishes running, IT RETURNS THE ADDRESS! NOT THE VARIABLE!
#SO BY THE TIME WE HAVE FINISHED ID , IT AS RELEASED ITS POINTER

1

In [75]:
# Its the same thing as doing this!

In [76]:
ref_count(2434899180744)

1

In [78]:
b = a# shared reference!

In [81]:
id(b) # same as "a"

2434899180744

In [83]:
ref_count(id(a))

2

In [92]:
c = a

In [93]:
ref_count(id(a))

2

In [94]:
c = 10

In [95]:
ref_count(id(a))  # Dropping "c"

1

In [88]:
b = None
ref_count(id(a)) # "b" is dropped

1

In [90]:
print(id(b)) # because 'b' i s None

1368368272


In [102]:
a_id = id(a)
a = None

ref_count(a_id)  # Whyt is this happening?

24408

In [100]:
ref_count(a_id)  # Whyt is this happening?

-420341279

In [103]:
ref_count(a_id)  # Whyt is this happening?

24407

In [104]:
# when last ref to that memory address is deleted ,
# memory manager frees up that address and something random pops up here!

### Garbage Collection

In [306]:
#As soon ref count is 0  python memory manager destroys object and reclaims the memory

In [307]:
# but its not so with circular references!
#circular references will lead to memory leak!

In [308]:
# Thats where garbage collector comes in 
# It will be able it identify circular references and delete it!

In [309]:
# It can be controlled programmatically using gc module!
# by default it is turned on
# you may turn it off if you're sure your code does not create circular references
# -but beware!
#be careful , this is more advanced!
#runs periodically on its own (if turned on)
# you can call it manually, and even do your own clean up!

In [310]:
#In general it works just fine 
# but not always
# especially  for Python <3.4
# even if one of the objects in circular references has a destructor , __del__()
# Then the order of objects may be important!
#but GC does not know what the order should be 
# So its marked as uncollectable!
#We would end up with a memory leak!

In [311]:
import ctypes
import gc  # garbage collection module

In [312]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [313]:
def object_by_id(object_id):
    for obj in gc.get_objects(): # every object in gc!
        if id(obj) == object_id:
            return "object exists"
        return " Not Found"

In [314]:
class A:
    def __init__(self):
        self.b = B(self)  #self.b is an instance of B()
        print('A: self: {0}, b: {1}'.format(hex(id(self)) ,hex(id(self.b))))

In [315]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)) ,hex(id(self.a))))
        #print(self.a)

In [316]:
#Circular reference complete!


In [317]:
gc.disable() # disabled garbage collector!

In [318]:
my_var = A()
#my_var is passed into Self( of A)
#my_var.b is an instance of Class B which has my_var as a parameter

B: self: 0x236eb5d2cc0, a: 0x236eb5d2c50
A: self: 0x236eb5d2c50, b: 0x236eb5d2cc0


In [319]:
hex(id(my_var))

'0x236eb5d2c50'

In [320]:
print(hex(id(my_var.b)))

0x236eb5d2cc0


In [321]:
print(hex(id(my_var.b.a)))

0x236eb5d2c50


In [322]:
a_id = id(my_var)
b_id = id(my_var.b)

In [323]:
print(hex(id(a_id)))
print(hex(id(b_id)))

0x236eb60e150
0x236eb5c3d50


In [324]:
ref_count(a_id)

2

In [325]:
ref_count(b_id)

1

In [326]:
object_by_id(a_id)

' Not Found'

In [327]:
object_by_id(b_id)

' Not Found'

In [330]:
my_var = None

In [331]:
ref_count(a_id)

1

In [332]:
ref_count(b_id)

1

In [335]:
gc.collect()

0

In [337]:
object_by_id(a_id)

' Not Found'

In [338]:
object_by_id(b_id)

' Not Found'

In [339]:
ref_count(a_id)

-1

In [341]:
ref_count(b_id)

-1

In [344]:
gc.enable()

In [345]:
gc.isenabled()

True

## Dynamic Typing vs Static Typing

In [346]:
#Some languages (Java, C++ ...) are statically typed

In [348]:
#String     myvar = 'hello'
#datatype   Variable  value     

In [349]:
#Data type is associated with the variable name!!

In [350]:
#i.e
# myvar = 10 does not work!

In [351]:
# This is not so in Python
# It is dynamically typed!

In [352]:
#my var is purely a reference to a string object with value hello
# No type is "attached to my_var"

In [354]:
my_var = 10
print(my_var)

10


In [356]:
my_var = 'hello'
print(my_var)

hello


In [357]:
# Did type of my var change?
# NO  my var did not even have a type to begin with!

In [358]:
# We can use type() to find the type of the object pointed at any point in the 
# execution

In [359]:
a = 'String'

In [360]:
type(a)

str

In [361]:
a = 10

In [362]:
type(a)

int

In [363]:
a = lambda x: x**2

In [367]:
type(a)

function

In [368]:
a = 3 + 4j

In [369]:
type(a)

complex

In [370]:
# Everytime we change the referance of object!

### Variable reassignment

In [375]:
a = 10

In [376]:
a = 15 
# We are not changing the value inside the object 10
# But instead a new object of 15 is created 
# And the reference is changed!

In [377]:
# Infact content of integets can never be changed!

In [378]:
hex(id(a))

'0x51966e00'

In [379]:
type(a)

int

In [380]:
a = 15

In [382]:
hex(id(a)) 
#Memory address of a has changed!

'0x51966e00'

In [383]:
print(type(a))

<class 'int'>


In [385]:
a = a +1
hex(id(a)) #Memory address changed again!!

'0x51966e40'

In [386]:
a = 10
b = 10


In [387]:
hex(id(a)) 


'0x51966d60'

In [388]:
hex(id(b)) 

'0x51966d60'

In [390]:
# WHY ARE "a" and "b" are pointing to the same object!???
# Why is python doing this??

### Object mutability

In [391]:
#Object has a 
    # Type
    # and state / data
    
# Changing the data inside the object is called modifying the internal state of the object

In [3]:
#Eg:  say a variable called my_account of a datatype BankAccount
# Bank account has two properties
# Account no say 123456
# Balance : 150

#Now if  we modify the Balance to 500
# We say that the internal state(data) has changed


# BUT THE MEMORY ADDRESS HAS NOT CHANGED

#Now we say that the object has MUTATED

In [4]:
# An object whose internal state can be changed is called Mutable


In [ ]:
# An object whose internal state cannot

In [6]:
#Immutable objects
# Numbers (int, float, Booleans, etc...)
# Strings
# Tuples
# Frozen Sets
# User Defined Class

In [8]:
# Mutable types
# Lists (We can Add remove and replace)
#Sets 
#Dictionaries 
#User-Defined Classes

A warning about mutability


In [9]:
t = (1,2,3)  # tuples are immutable:
#elements cannot be deleted, inserted or replaced
# and also elements inside tuple are integers which are immutable too

In [14]:
# Now consider this
a = [1, 2]
b = [3, 4]
t = ( a, b)  # Now lists are mutable 
t

([1, 2], [3, 4])

In [15]:
a.append(100)
b.append(500)

In [20]:
 t 
# Now we can add elements to "a" and "b"
# But the tuple in itself is immutable !
#t is mutable but element inside t is mutable
# But t  changed

#Not to confuse mutability with completely unchanged

#It's not absolutely fixed

([1, 2, 100], [3, 4, 500])

In [21]:
my_list = [1, 2, 3]

type(my_list)

list

In [22]:
hex(id(my_list))

'0x190ead502c8'

In [23]:
my_list.append(4)

In [24]:
my_list


[1, 2, 3, 4]

In [27]:
hex(id(my_list))  # Memory address is not changed even after appending!

'0x190ead502c8'

In [37]:
my_list1 = [1, 2, 3]

In [40]:
hex(id(my_list1)) # Even though my_list and my_List1 contains the same values they are 
# not the same objects

'0x190eadb4f88'

In [41]:
my_list1 = my_list1 + [4]

In [42]:
my_list1

[1, 2, 3, 4]

In [43]:
# We did'nt append here, we concatenate two objects here!
# python combined two objects and returned a new one!

In [45]:
hex(id(my_list1)) # The id is  not the same because its another object

'0x190eaddd848'

In [46]:
my_dict = dict(key1 =1,key2 = 5)

In [47]:
my_dict

{'key1': 1, 'key2': 5}

In [48]:
hex(id(my_dict))

'0x190eaddaab0'

In [51]:
my_dict['key3'] = 10.2531

In [52]:
my_dict


{'key1': 1, 'key2': 5, 'key3': 10.2531}

In [54]:
hex(id(my_dict)) # The address has not changed

'0x190eaddaab0'

In [55]:
t = (1, 2, 3)

In [56]:
t[1]

2

In [57]:
id(t[1])

1820879968

In [58]:
id(t[2])

1820880000

In [59]:
t = ([1, 2], [3, 4])

In [60]:
id(t)

1721925260360

In [65]:
t[1].append(3)
t  #element is mutable

([1, 2], [3, 4, 3, 3, 3])